In [1]:
# 2nd version
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from sklearn.impute import KNNImputer, SimpleImputer

In [2]:
mushroom_raw_df = pd.read_csv("/Users/eupirate/Desktop/CA_data_exp_visdual/mush.csv")
mushroom_raw_df.head(10)

,Mushroom ID,Cap Shape,Cap Color,Gill Size,Stalk Shape,Habitat,Edibility,Market Demand,Spore Print Color,Population,...,Stalk Surface Below Ring,Stalk Color Above Ring,Stalk Color Below Ring,Veil Type,Veil Color,Growth Rate,Inspection Date,Incubation date,Temperature,Measurement
0,1,sunken,brown,narrow,tapering,leaves,edible,1,orange,abundant,...,silky,white,brown,universal,orange,0.973096463,03/07/2021,03/02/2021,49,C
1,?,convex,brown,narrow,tapering,urban,edible,2,brown,several,...,scaly,pink,white,universal,white,?,03/05/2023,03/04/2023,13,F
2,3,flat,brown,broad,enlarging,grasses,edible,8,yellow,scattered,...,?,gray,orange,partial,orange,1.977619113,01/08/2022,01/04/2022,10,C
3,4,sunken,gray,narrow,tapering,?,poisonous,4,orange,scattered,...,scaly,gray,orange,universal,yellow,2.272762762,02/10/2023,02/02/2023,22,C
4,5,flat,red,?,enlarging,urban,edible,7,brown,several,...,fibrous,orange,white,partial,white,0.718873785,01/09/2023,01/04/2023,9,C
5,6,bell,?,narrow,tapering,grasses,poisonous,6,yellow,several,...,?,orange,white,universal,orange,14.02415593,04/11/2023,04/05/2023,21,C
6,7,flat,brown,broad,tapering,woods,?,5,yellow,scattered,...,scaly,white,white,universal,white,4.320348762,02/08/2023,02/04/2023,13,C
7,8,sunken,red,narrow,enlarging,urban,?,3,yellow,several,...,?,gray,orange,partial,yellow,1.450596518,02/07/2023,02/05/2023,2,C
8,9,?,yellow,narrow,?,grasses,edible,7,yellow,solitary,...,fibrous,brown,gray,?,orange,3.545415336,01/07/2023,01/01/2023,8,C
9,10,bell,?,broad,tapering,paths,poisonous,3,white,?,...,scaly,white,white,universal,yellow,3.866757255,04/05/2023,04/01/2023,10,C


In [3]:
# Handle Mushroom ID column indexing
mushroom_raw_df["Mushroom ID"] = mushroom_raw_df.index + 1
#print(mushroom_raw_df.iloc[:16])

In [4]:
# mushroom categorical columns
mushroom_raw_df.replace("?",np.nan,inplace=True)

# Include Edibility Column
'''
df_bool = mushroom_raw_df[["Cap Shape","Cap Color","Gill Size","Stalk Shape",
                            "Habitat","Edibility","Spore Print Color","Population",
                            "Odor","Bruising","Ring Type","Stalk Surface Above Ring","Stalk Surface Below Ring",
                            "Stalk Color Above Ring","Stalk Color Below Ring","Veil Type","Veil Color",
                            "Measurement"]].isna()
#print(df_bool)
df_bool.isnull().sum()
'''


# Exclude Edibility Column
df_bool = mushroom_raw_df[["Cap Shape","Cap Color","Gill Size","Stalk Shape",
                            "Habitat","Spore Print Color","Population",
                            "Odor","Bruising","Ring Type","Stalk Surface Above Ring","Stalk Surface Below Ring",
                            "Stalk Color Above Ring","Stalk Color Below Ring","Veil Type","Veil Color",
                            "Measurement"]].isna()
#print(df_bool)
df_bool.isnull().sum()

Cap Shape                   0
Cap Color                   0
Gill Size                   0
Stalk Shape                 0
Habitat                     0
Spore Print Color           0
Population                  0
Odor                        0
Bruising                    0
Ring Type                   0
Stalk Surface Above Ring    0
Stalk Surface Below Ring    0
Stalk Color Above Ring      0
Stalk Color Below Ring      0
Veil Type                   0
Veil Color                  0
Measurement                 0
dtype: int64

In [5]:
# include NaN, run describe()
mushroom_raw_df.describe()

,Mushroom ID,Temperature
count,1000.000000,1000.000000
mean,500.500000,14.666000
std,288.819436,8.698742
min,1.000000,0.000000
25%,250.750000,7.000000
50%,500.500000,15.000000
75%,750.250000,22.000000
max,1000.000000,49.000000


In [6]:
# now we drop all rows with any NaN values
mushroom_drop_NaN = mushroom_raw_df.dropna()
mushroom_drop_NaN.describe()

,Mushroom ID,Temperature
count,131.000000,131.000000
mean,524.580153,14.519084
std,298.132184,8.966048
min,1.000000,0.000000
25%,233.000000,7.500000
50%,544.000000,13.000000
75%,789.000000,22.500000
max,995.000000,49.000000


# Before dropping NaN, we had 1000 rows of data, after dropped NaN, we only have 119 rows of data to work with, which is not a good representation of the dataset. I think we should explore other approaches to replace with NaN values.

# Identified those columns are mushroom's feature categorical columns, we choose to apply mode to replace NaN with the most common value of a column

# With a better understanding of the data, Edibility column is excluded from the mode replacement, as we don't want to misclassify or make inaccurate predictions, we want to ensure predict mushroom edibility safely.

In [7]:
# Exclude Edibility Column, it will be handled seprately soon.
for column in mushroom_raw_df.select_dtypes(include="object").columns:
    if column != "Edibility":
        mushroom_raw_df[column].fillna(mushroom_raw_df[column].mode()[0], inplace=True)
#print(mushroom_raw_df)
#mushroom_raw_df.head(21)
#mushroom_raw_df.describe()

In [8]:
# make a copy of df - which excluded 'Edibility' column
mushroom_raw_df_tr=mushroom_raw_df.copy()
mushroom_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Mushroom ID               1000 non-null   int64 
 1   Cap Shape                 1000 non-null   object
 2   Cap Color                 1000 non-null   object
 3   Gill Size                 1000 non-null   object
 4   Stalk Shape               1000 non-null   object
 5   Habitat                   1000 non-null   object
 6   Edibility                 887 non-null    object
 7   Market Demand             1000 non-null   object
 8   Spore Print Color         1000 non-null   object
 9   Population                1000 non-null   object
 10  Odor                      1000 non-null   object
 11  Bruising                  1000 non-null   object
 12  Ring Type                 1000 non-null   object
 13  Stalk Surface Above Ring  1000 non-null   object
 14  Stalk Surface Below Ring 

In [9]:
# create a treated 'Edibility_Tr1' column hardcode "NotDefined"
'''
mushroom_raw_df_tr=mushroom_raw_df.copy()
mushroom_raw_df_tr["Edibility_Tr1"]=mushroom_raw_df_tr["Edibility"].fillna("NotDefined_1")
mushroom_raw_df_tr.info()
mushroom_raw_df_tr.head(21)
'''

'\nmushroom_raw_df_tr=mushroom_raw_df.copy()\nmushroom_raw_df_tr["Edibility_Tr1"]=mushroom_raw_df_tr["Edibility"].fillna("NotDefined_1")\nmushroom_raw_df_tr.info()\nmushroom_raw_df_tr.head(21)\n'

In [10]:
# Appy SimpleImputer
from sklearn.impute import SimpleImputer

mushroom_raw_df_tr=mushroom_raw_df.copy()

# Create "NotDefined" with SimpleImputer
imp=SimpleImputer(strategy="constant",fill_value="NotDefined_2")

# create a treated 'Edibility_Tr2' column by using SimpleImputer fit_transform function
mushroom_raw_df_tr[["Edibility_Tr2"]]=imp.fit_transform(mushroom_raw_df_tr[["Edibility"]])
mushroom_raw_df_tr[["Edibility_Tr2"]]
#mushroom_raw_df_tr.head(21)

,Edibility_Tr2
0,edible
1,edible
2,edible
3,poisonous
4,edible
...,...
995,poisonous
996,edible
997,edible
998,edible


In [ ]:
mushroom_raw_df_tr.groupby("Edibility_Tr2")["Habitat"]["Habitat"].sum()